<a href="https://colab.research.google.com/github/MutonyiLewis/DiabetesRecipes/blob/main/SimilaritywithGInLocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


##Import the dataset

In [ ]:
datapath = '/content/drive/MyDrive/Attachment Folder/RecipesWithLocationGI.csv'

##Import necessary libraries

In [ ]:
import pandas as pd

#NLTK contains english stopwords
!pip install nltk -q
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

#Library tokenizes words to tokens
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence


#create iterators for efficient looping.
import itertools


#Import tensorflow
!pip install --upgrade tensorflow_hub
import tensorflow_hub as hub #For embedddings

import numpy as np

#Read data from pdf files
# !pip install PyPDF2
# import PyPDF2

#Import cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

#import regex
import re



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##Read CSV Data

In [ ]:
df = pd.read_csv(datapath, encoding='cp1252')
df.head(5)

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),proteins(g),fibre(g),vitamin A(mcg),iron(mg),zinc(mg),F_factor_est,High GI(%),Low GI(%),Average GI(%),location
0,Kaimati(Fried Dumplings),15003,24,Kaimatis get their unique flavour from the sty...,"wheat flour, refined\nwater, vanilla essence,...",Put yeast in a small container.\n Add 50ml of...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,4.6,1.6,30,2.1,0.45,0.4,74.140807,33.147089,53.643948,Coast
1,Mahamri\n(Swahili Doughnut),15004,26,This is a typical traditional recipe among the...,"wheat flour,\ncoconut milk\nwhite sugar\ndry y...","Break the coconut shell, drain the water and...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,6.0,2.1,41,2.8,0.56,0.4,74.195795,34.506434,54.351114,Kenya
2,"Enriched Mandazi \n(East African Doughnuts, En...",15124,28,A popular snack among urban dwellers across th...,self-raising wheat flour\neggs\nmargarine\nRin...,"? Put flour, salt, sugar and lemon rind into ...","Energy 1,590 kJ/ 379 kcal | Fat 16.1 g | Carbo...",379.0,16.1,49.9,7.6,2.2,90,3.3,0.66,0.4,74.365585,39.479610,56.922598,Kenya
3,"Basic Mandazi \n(East African Doughnuts, Basic)",15125,30,You will find this recipe in any home across K...,all-purpose wheat flour\nbaking powder\nsugar\...,"? Put the wheat flour into a bowl, add baking...","Energy 1,430kJ/ 340 kcal | Fat 12.9 g | Carboh...",340.0,12.9,48.7,6.4,2.1,48,3.5,0.52,0.4,74.559279,44.016368,59.287824,Kenya
4,Meat Samosa\n(Sambusa ya Nyama),15025,32,Nothing more delicious like the Kenyan meaty s...,"minced beef\ncoriander, fresh\nleek\ngarlic\nc...",? Put the meat in a pan over a fire. Stir con...,"Energy 1,854 kJ/ 443 kcal | Fat 22.2 g | Carbo...",443.0,22.2,40.5,18.8,3.1,66,11.5,2.99,0.4,74.009986,31.775738,52.892862,Nairobi


##Cleaning the data

In [ ]:
stop = stopwords.words('english')

df['title'] = df['title'].str.replace('[^\w\s]', ' ', True)
df['title'] = df['title'].str.replace('\d+', '')
df['title'] = df['title'].str.replace('\n',' ')
df['title'] = df['title'].str.replace('()', '', True)
df['title'] = df['title'].str.lower()
df['ingridients'] = df['ingridients'].str.replace('[^\w\s]', ' ')
df['ingridients'] = df['ingridients'].str.replace('\n', ' ')
df['ingridients'] = df['ingridients'].str.replace('\d+', ' ')
df['ingridients'] = df['ingridients'].str.replace('\ ', ' ')
df['ingridients'] = df['ingridients'].str.replace('cups', ' ')
df['ingridients'] = df['ingridients'].str.replace('tbs', ' ')
df['ingridients'] = df['ingridients'].str.replace('tsp', ' ')
df['ingridients'] = df['ingridients'].str.replace('?', ' ')
df['ingridients'] = df['ingridients'].str.replace('()', '', True)


df['preparation'] = df['preparation'].str.replace('?', ' ')
df['about'] = df['about'].str.replace('?', ' ')
# Removing stop words
df['ingridients'] = df['ingridients'].apply(lambda item: " ". join(item for item in item.split() if item not in stop))
df['preparation'] = df['preparation'].apply(lambda item: " ". join(item for item in item.split() if item not in stop))
df['Index'] = df.index
df.head()

<ipython-input-5-85c08a97c208>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace('[^\w\s]', ' ', True)
<ipython-input-5-85c08a97c208>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace('\d+', '')
<ipython-input-5-85c08a97c208>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace('()', '', True)
<ipython-input-5-85c08a97c208>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ingridients'] = df['ingridients'].str.replace('[^\w\s]', ' ')
<ipython-input-5-85c08a97c208>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ingridients'] = df['ingridients'].str.replace('\d+', ' ')
<ipython-input-5-85c08a97c208>:11: FutureWarning: The defa

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),...,fibre(g),vitamin A(mcg),iron(mg),zinc(mg),F_factor_est,High GI(%),Low GI(%),Average GI(%),location,Index
0,kaimati fried dumplings),15003,24,Kaimatis get their unique flavour from the sty...,wheat flour refined water vanilla essence clea...,Put yeast small container. Add 50ml warmed wat...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,...,1.6,30,2.1,0.45,0.4,74.140807,33.147089,53.643948,Coast,0
1,mahamri swahili doughnut),15004,26,This is a typical traditional recipe among the...,wheat flour coconut milk white sugar dry yeast...,"Break coconut shell, drain water grate flesh. ...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,...,2.1,41,2.8,0.56,0.4,74.195795,34.506434,54.351114,Kenya,1
2,"enriched mandazi east african doughnuts, enr...",15124,28,A popular snack among urban dwellers across th...,self raising wheat flour eggs margarine Rind l...,"Put flour, salt, sugar lemon rind bowl mix. Mi...","Energy 1,590 kJ/ 379 kcal | Fat 16.1 g | Carbo...",379.0,16.1,49.9,...,2.2,90,3.3,0.66,0.4,74.365585,39.479610,56.922598,Kenya,2
3,"basic mandazi east african doughnuts, basic)",15125,30,You will find this recipe in any home across K...,purpose wheat flour baking powder sugar cookin...,"Put wheat flour bowl, add baking powder sugar....","Energy 1,430kJ/ 340 kcal | Fat 12.9 g | Carboh...",340.0,12.9,48.7,...,2.1,48,3.5,0.52,0.4,74.559279,44.016368,59.287824,Kenya,3
4,meat samosa sambusa ya nyama),15025,32,Nothing more delicious like the Kenyan meaty s...,minced beef coriander fresh leek garlic chilli...,Put meat pan fire. Stir continuously. When jui...,"Energy 1,854 kJ/ 443 kcal | Fat 22.2 g | Carbo...",443.0,22.2,40.5,...,3.1,66,11.5,2.99,0.4,74.009986,31.775738,52.892862,Nairobi,4


##Tokenize the title & Ingredients

In [ ]:
df['title_tokens'] = df['title'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))
#df['title_tokens'] = df['title'].apply(lambda x: x.split())
df['ingridient_tokens'] = df['ingridients'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))
#df['ingridient_tokens'] = df['ingridients'].apply(lambda x: x.split())
df.head(5)

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),...,iron(mg),zinc(mg),F_factor_est,High GI(%),Low GI(%),Average GI(%),location,Index,title_tokens,ingridient_tokens
0,kaimati fried dumplings),15003,24,Kaimatis get their unique flavour from the sty...,wheat flour refined water vanilla essence clea...,Put yeast small container. Add 50ml warmed wat...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,...,2.1,0.45,0.4,74.140807,33.147089,53.643948,Coast,0,kaimati fried dumplings),wheat flour refined water vanilla essence clea...
1,mahamri swahili doughnut),15004,26,This is a typical traditional recipe among the...,wheat flour coconut milk white sugar dry yeast...,"Break coconut shell, drain water grate flesh. ...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,...,2.8,0.56,0.4,74.195795,34.506434,54.351114,Kenya,1,mahamri swahili doughnut),wheat flour coconut milk white sugar dry yeast...
2,"enriched mandazi east african doughnuts, enr...",15124,28,A popular snack among urban dwellers across th...,self raising wheat flour eggs margarine Rind l...,"Put flour, salt, sugar lemon rind bowl mix. Mi...","Energy 1,590 kJ/ 379 kcal | Fat 16.1 g | Carbo...",379.0,16.1,49.9,...,3.3,0.66,0.4,74.365585,39.479610,56.922598,Kenya,2,"enriched mandazi east african doughnuts, enric...",self raising wheat flour eggs margarine Rind l...
3,"basic mandazi east african doughnuts, basic)",15125,30,You will find this recipe in any home across K...,purpose wheat flour baking powder sugar cookin...,"Put wheat flour bowl, add baking powder sugar....","Energy 1,430kJ/ 340 kcal | Fat 12.9 g | Carboh...",340.0,12.9,48.7,...,3.5,0.52,0.4,74.559279,44.016368,59.287824,Kenya,3,"basic mandazi east african doughnuts, basic)",purpose wheat flour baking powder sugar cookin...
4,meat samosa sambusa ya nyama),15025,32,Nothing more delicious like the Kenyan meaty s...,minced beef coriander fresh leek garlic chilli...,Put meat pan fire. Stir continuously. When jui...,"Energy 1,854 kJ/ 443 kcal | Fat 22.2 g | Carbo...",443.0,22.2,40.5,...,11.5,2.99,0.4,74.009986,31.775738,52.892862,Nairobi,4,meat samosa sambusa ya nyama),minced beef coriander fresh leek garlic chilli...


##Reclean tokens

In [ ]:
df['title_tokens'] = df['title_tokens'].str.replace('(', '', True)
df['title_tokens'] = df['title_tokens'].str.replace(')', '', True)

df['ingridient_tokens'] = df['ingridient_tokens'].str.replace('(', '', True)
df['ingridient_tokens'] = df['ingridient_tokens'].str.replace(')', '', True)
df.head(5)

<ipython-input-7-fff7d4207b3a>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['title_tokens'] = df['title_tokens'].str.replace('(', '', True)
<ipython-input-7-fff7d4207b3a>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['title_tokens'] = df['title_tokens'].str.replace(')', '', True)
<ipython-input-7-fff7d4207b3a>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['ingridient_tokens'] = df['ingridient_tokens'].str.replace('(', '', True)
<ipython-input-7-fff7d4207b3a>:5: FutureWarning: The default value of regex

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),...,iron(mg),zinc(mg),F_factor_est,High GI(%),Low GI(%),Average GI(%),location,Index,title_tokens,ingridient_tokens
0,kaimati fried dumplings),15003,24,Kaimatis get their unique flavour from the sty...,wheat flour refined water vanilla essence clea...,Put yeast small container. Add 50ml warmed wat...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,...,2.1,0.45,0.4,74.140807,33.147089,53.643948,Coast,0,kaimati fried dumplings,wheat flour refined water vanilla essence clea...
1,mahamri swahili doughnut),15004,26,This is a typical traditional recipe among the...,wheat flour coconut milk white sugar dry yeast...,"Break coconut shell, drain water grate flesh. ...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,...,2.8,0.56,0.4,74.195795,34.506434,54.351114,Kenya,1,mahamri swahili doughnut,wheat flour coconut milk white sugar dry yeast...
2,"enriched mandazi east african doughnuts, enr...",15124,28,A popular snack among urban dwellers across th...,self raising wheat flour eggs margarine Rind l...,"Put flour, salt, sugar lemon rind bowl mix. Mi...","Energy 1,590 kJ/ 379 kcal | Fat 16.1 g | Carbo...",379.0,16.1,49.9,...,3.3,0.66,0.4,74.365585,39.479610,56.922598,Kenya,2,"enriched mandazi east african doughnuts, enriched",self raising wheat flour eggs margarine Rind l...
3,"basic mandazi east african doughnuts, basic)",15125,30,You will find this recipe in any home across K...,purpose wheat flour baking powder sugar cookin...,"Put wheat flour bowl, add baking powder sugar....","Energy 1,430kJ/ 340 kcal | Fat 12.9 g | Carboh...",340.0,12.9,48.7,...,3.5,0.52,0.4,74.559279,44.016368,59.287824,Kenya,3,"basic mandazi east african doughnuts, basic",purpose wheat flour baking powder sugar cookin...
4,meat samosa sambusa ya nyama),15025,32,Nothing more delicious like the Kenyan meaty s...,minced beef coriander fresh leek garlic chilli...,Put meat pan fire. Stir continuously. When jui...,"Energy 1,854 kJ/ 443 kcal | Fat 22.2 g | Carbo...",443.0,22.2,40.5,...,11.5,2.99,0.4,74.009986,31.775738,52.892862,Nairobi,4,meat samosa sambusa ya nyama,minced beef coriander fresh leek garlic chilli...


##View the vocabulary size

In [ ]:
# # vocabtitle = list(itertools.chain.from_iterable(df['title_tokens']))
# # vocabtitle = list(set(vocabtitle))
# # print(vocabtitle)
# # print("Vocubalary size of the title tokens: " + str((len(vocabtitle))))

# # vocabingridients = list(itertools.chain.from_iterable(df['ingridient_tokens']))
# # vocabingridients = list(set(vocabingridients))
# # print("Vocubalary size of the ingridient tokens: " + str(len(vocabingridients)))
# !pip install hashable_df.cache()
# from hashable_df import hashable_df
# hashable_df(df).duplicated('title_tokens')

# my_list = tuple(df['title_tokens'])
# print(my_list)
# unique_words = set(my_list)
# vocabulary_size = len(unique_words)
# print(vocabulary_size)
# title_tokens = df['title'].apply(lambda x: x.split())
# title_tokens

# create vocabulary (optional)
title_vocab = list(itertools.chain.from_iterable(df['title_tokens'].apply(lambda x: x.split())))
title_vocab = list(set(title_vocab))
print(title_vocab)
print(len(title_vocab))

ingridient_vocab = list(itertools.chain.from_iterable(df['ingridient_tokens'].apply(lambda x: x.split())))
ingridient_vocab = list(set(ingridient_vocab))
print(ingridient_vocab)
print(len(ingridient_vocab))

['yoghurt', 'flour', 'porridge', 'millet,', 'gram', 'amaranth', 'mrenda', 'whole', 'likhanga', 'roti', 'viazi', 'hydrabadi', 'cha', 'milk', 'peas', 'bananas', 'ashir', 'wiki', 'kales', 'pilau', 'ndizi', 'wimbi', 'split', 'nderema', 'pumpkin', 'steamed', 'mushroom', 'diluted', 'butter', 'mayai', 'tsiswa', 'termites', 'siro', 'boiled', 'nzenga', 'brown', 'kidney', 'plain', 'kuchemshwa', 'bhajia', 'sweetened', 'dal', 'coconut', 'samosa', 'kimanga', 'mboga', 'fish', 'white', 'raw', 'mbuboyi', 'finger', 'sauted', 'yasmin', 'bhature', 'quails', 'doughnuts,', 'kamande', 'sorghum', 'nightshade', 'east', 'beef,', 'ndengu', 'sweet', 'stir-fried', 'njahi/dolichos', 'malesele', 'mawele', 'seveve', 'kuta', 'dried', 'ogkole', 'sardine', 'mahindi', 'pancake', 'kaimati', 'lentil', 'fiqe', 'mahamri', 'millet', 'bean', 'nchenga', 'silver', 'red', 'african', 'thaa', 'potatoes', 'enriched', 'stew', 'blood', 'drop', 'banana', 'goat', 'pan', 'mandazi', 'rhikhuvi', 'onion', 'beans,', 'perch', 'pigeon', 'maji

##Vectorize(encode) the generated tokens

In [ ]:
##Using Universal sentence encoder
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")

#generate embeddings
title_embeddings = embed(df['title_tokens'])
ingredient_embeddings = embed(df['ingridient_tokens'])


#create list from np arrays
use_title= np.array(title_embeddings).tolist()
use_ingredients= np.array(ingredient_embeddings).tolist()


#add lists as dataframe column
df['use_title_tokens'] = use_title
df['use_ingredient_tokens'] = use_ingredients


#check dataframe
df.head(2)

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),...,F_factor_est,High GI(%),Low GI(%),Average GI(%),location,Index,title_tokens,ingridient_tokens,use_title_tokens,use_ingredient_tokens
0,kaimati fried dumplings),15003,24,Kaimatis get their unique flavour from the sty...,wheat flour refined water vanilla essence clea...,Put yeast small container. Add 50ml warmed wat...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,...,0.4,74.140807,33.147089,53.643948,Coast,0,kaimati fried dumplings,wheat flour refined water vanilla essence clea...,"[0.018677085638046265, 0.027695173397660255, 0...","[0.05093974620103836, -0.059134338051080704, -..."
1,mahamri swahili doughnut),15004,26,This is a typical traditional recipe among the...,wheat flour coconut milk white sugar dry yeast...,"Break coconut shell, drain water grate flesh. ...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,...,0.4,74.195795,34.506434,54.351114,Kenya,1,mahamri swahili doughnut,wheat flour coconut milk white sugar dry yeast...,"[0.05975484475493431, 0.014897819608449936, -0...","[0.055805377662181854, -0.008994420990347862, ..."


##Appling Cosine similarity for patient preference on food using title


In [ ]:
#Get a sample input
sample_input_title = df['title_tokens'].iloc[50]
sample_input_title = [sample_input_title]
sample_input_title

#generate sample's embeddings
sample_embeddings = embed(sample_input_title)
#create list from np arrays
sample_use_title = np.array(sample_embeddings).tolist()
#add lists as dataframe column
sample_feature_title = sample_use_title

print(sample_feature_title)


[[0.002496495610103011, 0.00021615692821796983, 0.025483673438429832, -0.03950144723057747, 0.03191708028316498, -0.06057032570242882, 0.019827034324407578, 0.0123374555259943, -0.06490050256252289, 0.072123222053051, 0.0008730001281946898, 0.04941543564200401, -0.005297477822750807, -0.013068951666355133, -0.07933781296014786, 0.04259758070111275, -0.0753253698348999, 0.04045233502984047, -0.0777154341340065, -0.07364583015441895, 0.054841816425323486, 0.028268972411751747, -0.04652786999940872, 0.04365582391619682, -0.06156441941857338, 0.07492231577634811, -0.04607005789875984, 0.046329040080308914, -0.0014796142932027578, 0.0670173168182373, 0.04143005609512329, -0.0011447665747255087, 0.0603475496172905, 0.06583639979362488, -0.029031626880168915, 0.07877296954393387, 0.009142833761870861, 0.058224111795425415, -0.044941309839487076, -0.04846222326159477, 0.059275053441524506, -0.03646416589617729, 0.05839686468243599, 0.022923026233911514, -0.07692436128854752, 0.0245008151978254

In [ ]:
##Apply it to the dataset
df['sample_use_title_similarity'] = df['use_title_tokens'].apply(lambda x: cosine_similarity([x], sample_feature_title)[0][0])
df.head(5)

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),...,High GI(%),Low GI(%),Average GI(%),location,Index,title_tokens,ingridient_tokens,use_title_tokens,use_ingredient_tokens,sample_use_title_similarity
0,kaimati fried dumplings),15003,24,Kaimatis get their unique flavour from the sty...,wheat flour refined water vanilla essence clea...,Put yeast small container. Add 50ml warmed wat...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,...,74.140807,33.147089,53.643948,Coast,0,kaimati fried dumplings,wheat flour refined water vanilla essence clea...,"[0.018677085638046265, 0.027695173397660255, 0...","[0.05093974620103836, -0.059134338051080704, -...",0.282716
1,mahamri swahili doughnut),15004,26,This is a typical traditional recipe among the...,wheat flour coconut milk white sugar dry yeast...,"Break coconut shell, drain water grate flesh. ...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,...,74.195795,34.506434,54.351114,Kenya,1,mahamri swahili doughnut,wheat flour coconut milk white sugar dry yeast...,"[0.05975484475493431, 0.014897819608449936, -0...","[0.055805377662181854, -0.008994420990347862, ...",0.203635
2,"enriched mandazi east african doughnuts, enr...",15124,28,A popular snack among urban dwellers across th...,self raising wheat flour eggs margarine Rind l...,"Put flour, salt, sugar lemon rind bowl mix. Mi...","Energy 1,590 kJ/ 379 kcal | Fat 16.1 g | Carbo...",379.0,16.1,49.9,...,74.365585,39.479610,56.922598,Kenya,2,"enriched mandazi east african doughnuts, enriched",self raising wheat flour eggs margarine Rind l...,"[0.05596411973237991, 0.023343952372670174, -0...","[0.04107295721769333, -0.011059963144361973, 0...",0.237610
3,"basic mandazi east african doughnuts, basic)",15125,30,You will find this recipe in any home across K...,purpose wheat flour baking powder sugar cookin...,"Put wheat flour bowl, add baking powder sugar....","Energy 1,430kJ/ 340 kcal | Fat 12.9 g | Carboh...",340.0,12.9,48.7,...,74.559279,44.016368,59.287824,Kenya,3,"basic mandazi east african doughnuts, basic",purpose wheat flour baking powder sugar cookin...,"[-0.012403649277985096, -0.025107331573963165,...","[0.0600864477455616, -0.01692269556224346, -0....",0.246410
4,meat samosa sambusa ya nyama),15025,32,Nothing more delicious like the Kenyan meaty s...,minced beef coriander fresh leek garlic chilli...,Put meat pan fire. Stir continuously. When jui...,"Energy 1,854 kJ/ 443 kcal | Fat 22.2 g | Carbo...",443.0,22.2,40.5,...,74.009986,31.775738,52.892862,Nairobi,4,meat samosa sambusa ya nyama,minced beef coriander fresh leek garlic chilli...,"[0.03849627822637558, 0.007602210622280836, 0....","[0.05790663883090019, -0.035895101726055145, 0...",0.291942


In [ ]:
sorted_sample_title = df.sort_values(by='sample_use_title_similarity', ascending=False)
sorted_sample_title.head(3)

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),...,High GI(%),Low GI(%),Average GI(%),location,Index,title_tokens,ingridient_tokens,use_title_tokens,use_ingredient_tokens,sample_use_title_similarity
50,nyenyi mashed pigeon peas & green maize),15118,130,Nyenyi is a traditional delicacy among the E...,pigeon peas potatoes pumpkin leaves maize cobs...,Preparation 10 minutes | Cooking 3 hours | Ser...,Energy 524 kJ/ 125 kcal | Fat 2.3 g | Carbohyd...,125.0,2.3,17.8,...,75.455553,66.860464,71.158008,Central,50,nyenyi mashed pigeon peas & green maize,pigeon peas potatoes pumpkin leaves maize cobs...,"[0.0024965035263448954, 0.00021616635785903782...","[-0.03742131590843201, 0.00546188373118639, 0....",1.000000
42,kimanga cha mihogo mashed cassava and pigeon...,15084,114,This is a variant of the traditional Kimanga a...,cassava pigeon peas salt water,Preparation 10 minutes | Cooking 2 hours 45min...,Energy 823 kJ/ 194 kcal | Fat 0.6 g | Carbohyd...,194.0,0.6,39.6,...,75.157282,61.163808,68.160545,Coast,42,kimanga cha mihogo mashed cassava pigeon peas,cassava pigeon peas salt water,"[0.026489628478884697, 0.030509455129504204, 0...","[0.03539447858929634, 0.039454251527786255, 0....",0.788591
62,mchuzi wa mbaazi pigeon peas stew),15092,158,"Commonly known as mbaazi stew, this recipe i...",pigeon pea onion tomatoes cooking fat salt water,Preparation 10 minutes | Cooking 2 hours 40 mi...,Energy 494 kJ/ 118 kcal | Fat 4.4 g | Carbohy...,118.0,4.4,11.0,...,75.491386,66.823575,71.157480,Eastern,62,mchuzi wa mbaazi pigeon peas stew,pigeon pea onion tomatoes cooking fat salt water,"[0.02097218856215477, 0.03326470032334328, 0.0...","[0.023475168272852898, 0.024585489183664322, 0...",0.696320


##Function that returns most similar recipes

In [ ]:
def returnRecipetoTitle (query: str,location:str,blood_sugar:float):
    #Blood sugar in mmol/L
    if (blood_sugar <= 5.4):
      #Can recommend to eat all foods (Normal blood sugar)
      query = [query]
      location = location
      embeddings = embed(query)
      title_use = np.array(embeddings).tolist()
      input_features = title_use
      df['use_title_similarity'] = df['use_title_tokens'].apply(lambda x: cosine_similarity([x], input_features)[0][0])
      sorted_sample = df.sort_values(by='use_title_similarity', ascending=False)
      # result = sorted_sample['name'],sorted_sample['ingredients'],sorted_sample['steps'].head(10)
      ##Location section
      # for i in sorted_sample['location']:
      #   if (i == location):
      result = sorted_sample[['title', 'ingridients', 'preparation', 'carbohydrates(g)', 'proteins(g)', 'fibre(g)', 'location']].head(1).to_dict('records')
      result = result[0] # Access the first (and only) dictionary
      #    break
      #result = sorted_sample['title'].head(5)
    elif (blood_sugar<= 6.9):
      #can recommend foods less than 65 GI (Prediabetic)
      df_filtered = df[df['averageGI'] < 65].copy()  # Filter dataframe for GI < 65

      # Perform similarity calculations and sorting on the filtered dataframe
      query = [query]
      location = location
      embeddings = embed(query)
      title_use = np.array(embeddings)
      input_features = title_use
      df_filtered['use_title_similarity'] = df_filtered['use_title_tokens'].apply(
           lambda x: cosine_similarity([x], input_features)[0][0]
       )
      sorted_sample = df_filtered.sort_values(by='use_title_similarity', ascending=False)

      # Find recommendations based on location
      # for i in sorted_sample['location']:
      #     if (i == location):
      result = sorted_sample[['title', 'ingridients', 'preparation', 'carbohydrates(g)', 'proteins(g)', 'fibre(g)', 'location']].head(1).to_dict('records')
      result = result[0] # Access the first (and only) dictionary
      #        break  # Exit the loop once matching location is found
    else:
      #Recommend food with GI less than 55 GI (Diabetic)
      df_filtered = df[df['averageGI'] < 55].copy()  # Filter dataframe for GI < 55

      # Perform similarity calculations and sorting on the filtered dataframe
      query = [query]
      location = location
      embeddings = embed(query)
      title_use = np.array(embeddings).tolist()
      input_features = title_use
      df_filtered['use_title_similarity'] = df_filtered['use_title_tokens'].apply(
           lambda x: cosine_similarity([x], input_features)[0][0]
       )
      sorted_sample = df_filtered.sort_values(by='use_title_similarity', ascending=False)

      # # Find recommendations based on location
      # for i in sorted_sample['location']:
      #     if (i == location):
      result = sorted_sample[['title', 'ingridients', 'preparation', 'carbohydrates(g)', 'proteins(g)', 'fibre(g)', 'location']].head(1).to_dict('records')
      result = result[0] # Access the first (and only) dictionary
      #        break  # Exit the loop once matching location is found

    return result

##ORG code


  # query = [query]
  # embeddings = embed(query)
  # use = np.array(embeddings).tolist()
  # input_features = use
  # df['useSimilarity'] = df['useTokens'].apply(lambda x: cosine_similarity([x], input_features)[0][0])
  # sorted_sample = df.sort_values(by='useSimilarity', ascending=False)
  # # result = sorted_sample['name'],sorted_sample['ingredients'],sorted_sample['steps'].head(10)
  # result = sorted_sample['page'].head(1)

##Code end

  # arr = list(result)

  # for i in result:
  #   recipes = sorted_sample[sorted_sample['id' == i]]
  #   recipe_dict = dict(recipes)
  # importing required modules

  #return list(result)

In [ ]:
# my_title = ReturnTitle()
returnRecipetoTitle("sweet potato", "Nairobi", 5)

{'title': 'kitojo  sweet potato leaves)',
 'ingridients': 'sweet potato leaves onion cooking oil salt',
 'preparation': 'Preparation time 5-10 | Cooking time 15 minutes | Serves 4 Remove leaves vines wash thor\x02oughly. Cut leaves small strips. Prepare chop onion. Add cooking oil pot, add chopped onions fry golden brown. Add vegetables salt. Stir cook ready soft. Serve.',
 'carbohydrates(g)': 1.4,
 'proteins(g)': 3.9,
 'fibre(g)': 4.6,
 'location': 'Coast'}

##Gradio interface

In [ ]:
# #Install and
# !pip install -q gradio
# !pip install typing_extensions


In [ ]:
# import typing_extensions
# !pip show typing-extensions

In [ ]:
# import gradio as gr

# def greet(name):
#     return "Hello " + name + "!"

# demo = gr.Interface(fn=greet, inputs="textbox", outputs="textbox")

# if __name__ == "__main__":
#     demo.launch()


##A simple voila interface

In [ ]:
!pip install -q voila

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.2/380.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.0 which is incompatible.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.2 which 

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
locations = ["Kenya", "Nairobi", "Nakuru"]


query_input = widgets.Text(description="Query:")
location_input = widgets.Dropdown(options=locations, value="Nairobi", description="Location:")
blood_sugar_input = widgets.FloatSlider(min=0, max=20, step=0.1, value=5.0, description="Blood Sugar (mmol/L):")

# Button to trigger the function call
button = widgets.Button(description="Get Recipe Recommendations")

In [ ]:
def on_button_click(b):
    with output:
        output.clear_output()
        query = query_input.value
        location = location_input.value
        blood_sugar = blood_sugar_input.value

        recommendations = returnRecipetoTitle(query, location, blood_sugar)

        # Display the recommendations (example using a list)
        display(widgets.HTML(value=f"<h4>Recommended Recipes:</h4><ul>{''.join(f'<li>{rec}</li>' for rec in recommendations)}</ul>"))


In [ ]:
button.on_click(on_button_click)


In [ ]:
output = widgets.Output()  # To display output
display(widgets.VBox([query_input, location_input, blood_sugar_input, button, output]))


##Using Flask and ngrok

In [ ]:
!pip install Flask
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!pip install flask_cors
!ngrok authtoken 2C7jsuCluQvyGPsL0OwDfCQpE4y_3Ty2m8P8zeddGanp7ZPNi

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=fa5651375bf74608900d731b1f4b86abda8c8c41f8604319ebda60d6a529a9ee
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
import numpy as np
from flask_cors import CORS
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*", "methods": "*"}})

@app.route("/get_recipe_recommendations", methods=["POST"])
def get_recipe_recommendations():
    data = request.get_json()
    query = data.get("query")
    location = data.get("location")
    blood_sugar = float(data.get("blood_sugar"))

    recommendations = returnRecipetoTitle(query, location, blood_sugar)
    return jsonify({"recommendations": recommendations})


@app.route("/get_home_recipes", methods=["GET"])
def get_homescreen_recipes():
  pass
# Run the app with ngrok
from flask_ngrok import run_with_ngrok
run_with_ngrok(app)

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d201-35-188-226-133.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:04:19] "POST /get_recipe_recommendations HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:04:47] "POST /get_recipe_recommendations HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:08:52] "POST /get_recipe_recommendations HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:09:52] "POST /get_recipe_recommendations HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:10:45] "POST /get_recipe_recommendations HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:10:52] "POST /get_recipe_recommendations HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:14:45] "POST /get_recipe_recommendations HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:15:56] "POST /get_recipe_recommendations HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:18:58] "POST /get_recipe_recommendations HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2024 19:49:39] "POST /get_recipe_recommendations HTTP/1

In [ ]:
# !lsof -i :5000